In [52]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
from dash import dash_table
import io
import base64
import numpy as np

#Loading and preparing the Data

empl_data = pd.read_csv("Cleaned_EmployeeAttrition.csv")

dept = sorted(empl_data["Department"].unique())
jb_roles = sorted(empl_data["JobRole"].unique())
attr_val = sorted(empl_data["Attrition"].unique())

num_col = empl_data.select_dtypes(include=[np.number]).columns.tolist()

#Function for creating KPI cards for summary statistics

def kpi_crds(df):
    t_epmls = len(df)
    avg_inc = df["MonthlyIncome"].mean() if t_epmls > 0 else 0
    at_count = df["Attrition"].value_counts().to_dict()
    yes_count = at_count.get("Yes", 0)
    at_rate = (yes_count / t_epmls * 100) if t_epmls > 0 else 0
    crd_style = {"height": "100%", "textAlign": "center"}
    return dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Employees", className="card-title"),
                    html.H2(f"{t_epmls}", className="card-text text-primary")
                ])
            ], style=crd_style), md=4
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H4("Avg Monthly Income", className="card-title"),
                    html.H2(f"${avg_inc:,.2f}", className="card-text text-success")
                ])
            ], style=crd_style), md=4
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H4("Attrition Rate", className="card-title"),
                    html.H2(f"{at_rate:.2f}%", className="card-text text-danger")
                ])
            ], style=crd_style), md=4
        )
    ], className="mb-4")

#scatter plot for monthly income vs. tenure
def plot_inc_tenure(df):
    fig = px.scatter(df, x="YearsAtCompany", y="MonthlyIncome", color="Attrition",
                     hover_data=["Age", "JobRole"], title="Income vs. Years at Company")
    fig.update_layout(transition_duration=500)
    return fig

# bar chart for attrition counts by department
def plot_dept(df):
    counts = df.groupby(["Department", "Attrition"]).size().reset_index(name='Count')
    fig = px.bar(counts, x="Department", y="Count", color="Attrition",
                 barmode='group', title="Attrition Counts by Department")
    fig.update_layout(transition_duration=500)
    return fig

#heatmap for the correlation matrix
def heatmap_plot(df):
    if df.empty:
        corr = empl_data[num_col].corr()
    else:
        corr = df[num_col].corr()
    fig = go.Figure(data=go.Heatmap(
        z=corr.values, x=corr.columns, y=corr.columns,
        colorscale='Teal', text=corr.round(2).values, texttemplate='%{text}'
    ))
    fig.update_layout(title="Correlation Matrix")
    return fig

#trend chart for income over tenure
def plot_trend_chart(df, chart_type):
    trend_data = df.groupby("YearsAtCompany")["MonthlyIncome"].mean().reset_index()
    if chart_type == 'line':
        fig = px.line(trend_data, x="YearsAtCompany", y="MonthlyIncome", title="Average Monthly Income Over Tenure")
    else:
        fig = px.bar(trend_data, x="YearsAtCompany", y="MonthlyIncome", title="Average Monthly Income Over Tenure")
    fig.update_layout(transition_duration=500)
    return fig

#histogram for monthly income distribution
def inc_distribution_plot(df):
    fig = px.histogram(df, x="MonthlyIncome", color="Attrition", nbins=20,
                       title="Monthly Income Distribution")
    fig.update_layout(transition_duration=500)
    return fig

#A Funtion for filtering data based on selected criteria
def fltr_data(dp_select, rol_select, att_select, inc_range):
    df = empl_data.copy()
    if dp_select:
        df = df[df['Department'].isin(dp_select)]
    if rol_select:
        df = df[df['JobRole'].isin(rol_select)]
    if att_select:
        df = df[df['Attrition'].isin(att_select)]
    if inc_range:
        df = df[(df['MonthlyIncome'] >= inc_range[0]) & (df['MonthlyIncome'] <= inc_range[1])]
    return df

def dwnld_csv(data):
    df = pd.DataFrame(data)
    csv_string = df.to_csv(index=False)
    b64 = base64.b64encode(csv_string.encode()).decode()
    return f"data:text/csv;base64,{b64}"

#Layouts


def hero_header():
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Learn More About Employee Attrition", className="text-center mb-4", style={"color": "#0B3D91"}),
                html.P("Optimize your search with filters, explore patterns via charts, and download filtered data for further analysis.",
                       className="text-center", style={"fontSize": "1.2rem"})
            ])
        ])
    ], className="py-3 mb-4", style={"backgroundColor": "#e9f5ff", "borderRadius": "5px"})

def navbar():
    return dbc.Navbar(
        dbc.Container([
            dbc.NavbarBrand("Employee Attrition Dashboard", className="me-auto", style={"fontWeight": "bold"}),
            html.Img(src="/Emp_attrition.png", height="40px")
        ]),
        color="primary",
        dark=True,
        className="mb-3"
    )

def sidebar_filters():
    return dbc.Card([
        dbc.CardHeader("Filters & Controls"),
        dbc.CardBody([
            html.Label("Department"),
            dcc.Dropdown(
                id='filter-dept',
                options=[{'label': d, 'value': d} for d in dept],
                multi=True,
                placeholder="Select Department(s)"
            ),
            html.Br(),
            html.Label("Job Role (Searchable)"),
            dcc.Dropdown(
                id='filter-job',
                options=[{'label': r, 'value': r} for r in jb_roles],
                multi=True,
                placeholder="Select Job Role(s)",
                searchable=True
            ),
            html.Br(),
            html.Label("Attrition"),
            dcc.Dropdown(
                id='filter-attr',
                options=[{'label': a, 'value': a} for a in attr_val],
                multi=True,
                placeholder="Select Attrition Status"
            ),
            html.Br(),
            html.Label("Monthly Income Range"),
            dcc.RangeSlider(
                id='filter-income-range',
                min=int(empl_data["MonthlyIncome"].min()),
                max=int(empl_data["MonthlyIncome"].max()),
                value=[int(empl_data["MonthlyIncome"].min()), int(empl_data["MonthlyIncome"].max())],
                marks=None,
                tooltip={"placement": "bottom", "always_visible": True}
            ),
            html.Br(),
            dbc.Button("Download Filtered Data", id="download-btn", color="success", className="me-2"),
            dcc.Download(id="download-dataframe-csv"),
        ])
    ], className="mb-4")


def instructions_accordion():
    return dbc.Accordion([
        dbc.AccordionItem([
            html.P("Staff can be filtered by department, role, attrition status, and monthly income range using the left-hand filters."),
            html.P("Switch between tabs to find various insights:"),
            html.Ul([
                html.Li("Overview: Basic departmental attrition breakdown and quick KPIs."),
                html.Li("Findings: Analyze the distribution of income, the relationships between attributes, and the patterns of individual scatter."),
                html.Li("Trends: View the line or bar representation of the average monthly salary as it changes during the years of employment with the company.")
            ])
        ], title="How to Use")
    ], className="mb-4")

#Application Installation.

app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])
app.title = "Employee Attrition-Analytics"
app.layout = dbc.Container(fluid=True, children=[
    navbar(),
    hero_header(),
    dbc.Row([
        dbc.Col([
            sidebar_filters(),
            instructions_accordion()
        ], md=3),
        dbc.Col([
            html.Div(id='kpi-section'),
            dcc.Tabs(id="main-tabs", value='tab-overview', children=[
                dcc.Tab(label='Overview', value='tab-overview', children=[
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='chart-attr-dept'), md=12),
                    ])
                ]),
                dcc.Tab(label='Insights', value='tab-insights', children=[
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='chart-income-tenure'), md=6),
                        dbc.Col(dcc.Graph(id='chart-income-dist'), md=6)
                    ], className="mb-4"),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='chart-corr'), md=12)
                    ])
                ]),
                dcc.Tab(label='Trends', value='tab-trends', children=[
                    dbc.Row([
                        dbc.Col([
                            html.Label("Select Chart Type"),
                            dcc.RadioItems(
                                id='trend-chart-type',
                                options=[{"label": "Line", "value": "line"}, {"label": "Bar", "value": "bar"}],
                                value="line",
                                inline=True
                            ),
                            dcc.Graph(id='chart-trend')
                        ], md=12)
                    ])
                ]),
            ]),
            html.Hr(),
            html.H3("Filtered Employee Records", className="my-3"),
            dash_table.DataTable(
                id='employee-table',
                columns=[{"name": i, "id": i} for i in empl_data.columns],
                page_size=10,
                style_table={'overflowX': 'auto'},
                style_cell={'whiteSpace': 'normal', 'height': 'auto'}
            )
        ], md=9)
    ])
])

@app.callback(
    Output('kpi-section', 'children'),Output('employee-table', 'data'),Output('chart-attr-dept', 'figure'),
    Output('chart-income-tenure', 'figure'),Output('chart-income-dist', 'figure'),Output('chart-corr', 'figure'),
    Output('chart-trend', 'figure'),Input('filter-dept', 'value'),Input('filter-job', 'value'),Input('filter-attr', 'value'),
    Input('filter-income-range', 'value'),Input('trend-chart-type', 'value')
)

def update_dashboard(dp_select, job_sel, att_select, income_rng, trend_type):
    df = fltr_data(dp_select, job_sel, att_select, income_rng)
    kpis = kpi_crds(df)
    table_data = df.to_dict('records')

    fig_attr_dept = plot_dept(df)
    fig_income_tenure = plot_inc_tenure(df)
    fig_income_dist = inc_distribution_plot(df)
    fig_corr = heatmap_plot(df)
    fig_trend = plot_trend_chart(df, trend_type)
    return kpis, table_data, fig_attr_dept, fig_income_tenure, fig_income_dist, fig_corr, fig_trend
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-btn", "n_clicks"),
    State('employee-table', 'data'),
    prevent_initial_call=True
)
def download_filtered_data(n_clicks, table_data):
    return dict(content=pd.DataFrame(table_data).to_csv(index=False), filename="filtered_empl_data.csv")

# Run the Server
if __name__ == '__main__':
    app.run_server(debug=True)